# Homework 1: Sine wave generation and binary classification

## Part A - Sine Wave Generation

### Setup
To complete this part, install the required Python libraries:

In [4]:
# comment this out when submitting to gradescope
!pip install mido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.2 MB/s eta 0:00:00


In [5]:
import numpy as np
from scipy.io import wavfile

import numpy as np
import glob
from mido import MidiFile
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import math

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [6]:
# (installation process may be different on your system)
# You don't need to use these libraries, so long as you implement the specified functions
# !pip install numpy
# !pip install scipy
# !pip install IPython
# !pip install glob
# !pip install scikit-learn
# !pip install mido

1. Write a function that converts a musical note name to its corresponding frequency in Hertz (Hz)

`note_name_to_frequency()`
- **Input**: A string `note_name` combining a note (e.g., `'C'`, `'C#'`, `'D'`, `'D#'`, `'E'`, `'F'`, `'F#'`, `'G'`, `'G#'`, `'A'`, `'A#'`, `'B'`) and an octave number (`'0'` to `'10'`)
- **Output**: A float representing the frequency in Hz
- **Details**:
  - Use A4 = 440 Hz as the reference frequency
  - Frequencies double with each octave increase (e.g., A5 = 880 Hz) and halve with each decrease (e.g., A3 = 220 Hz)

- **Examples**:
  - `'A4'` → `440.0`
  - `'A3'` → `220.0`
  - `'G#4'` → `415.3047`

In [7]:
SAMPLE_RATE = 44100

def note_name_to_frequency(note_name):
    # Q1: Your code goes here

    # parse given note_name
    octave = int(note_name[-1])
    note = note_name[:-1]
    if note_name[-2:] == '10':
      octave = 10
      note = note_name[:-2]

    # find MIDI numbers
    notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    A4_midi = 69
    note_midi = (octave + 1) * len(notes) + notes.index(note)

    # calculate frequency
    frequency = 440 * 2 ** ((note_midi - A4_midi) / 12)

    return frequency

In [8]:
# sanity check
print(note_name_to_frequency('A4'))
print(note_name_to_frequency('A3'))
print(note_name_to_frequency('G#4'))

440.0
220.0
415.3046975799451


2. Write a function that linearly decreases the amplitude of a given waveform

`decrease_amplitude()`
- **Inputs**:
  - `audio`: A NumPy array representing the audio waveform at a sample rate of 44100 Hz
- **Output**: A NumPy array representing the audio waveform at a sample rate of 44100 Hz
- **Details**:
  - The function must linearly decrease the amplitude of the input audio. The amplitude should start at 1 (full volume) and decrease gradually to 0 (silence) by the end of the sample

In [9]:
def decrease_amplitude(audio):
    # Q2: Your code goes here

    # linspace: https://numpy.org/doc/stable/reference/generated/numpy.linspace.html
    multiplier = np.linspace(1, 0, len(audio))

    return audio * multiplier

3. Write a function that adds a delay effect to a given audio where the output is a combination of the original audio and a delayed audio

`add_delay_effects()`  
- **Inputs**:  
  - `audio`: A NumPy array representing the audio waveform, sampled at 44,100 Hz
- **Output**:  
  - A NumPy array representing the modified audio waveform, sampled at 44,100 Hz
- **Details**:
  - The amplitude of the delayed audio should be 30% of the original audio's amplitude
  - The amplitude of the original audio should be adjusted to 70% of the original audio's amplitude
  - The output should combine the original audio (with the adjusted amplitude) with a delayed version of itself
  - The delayed audio should be offset by 0.5 seconds behind the original audio

- **Examples**:
  - The provided files (input.wav and output.wav) provide examples of input and output audio

In [10]:
# Can use these for visualization if you like, though the autograder won't use ipython
#
# from IPython.display import Audio, display
#
# print("Example Input Audio:")
# display(Audio(filename = "input.wav", rate=44100))
#
# print("Example Output Audio:")
# display(Audio(filename = "output.wav", rate=44100))

In [16]:
def add_delay_effects(audio):
    #Q3: Your code goes here
    original = np.concatenate([audio * 0.7, np.zeros(int(0.5 * SAMPLE_RATE))])
    delayed = np.concatenate([np.zeros(int(0.5 * SAMPLE_RATE)), audio * 0.3])
    delayed_audio = original + delayed

    return delayed_audio

4. Write a function that concatenates a list of audio arrays sequentially and a function that mixes audio arrays by scaling and summing them, simulating simultaneous playback

`concatenate_audio()`
- **Input**:
  - `list_of_your_audio`: A list of NumPy arrays (e.g., `[audio1, audio2]`), each representing audio at 44100 Hz
- **Output**: A NumPy array of the concatenated audio
- **Example**:
  - If `audio1` is 2 seconds (88200 samples) and `audio2` is 1 second (44100 samples), the output is 3 seconds (132300 samples)

`mix_audio()`
- **Inputs**:
  - `list_of_your_audio`: A list of NumPy arrays (e.g., `[audio1, audio2]`), all with the same length at 44100 Hz.
  - `amplitudes`: A list of floats (e.g., `[0.2, 0.8]`) matching the length of `list_of_your_audio`
- **Output**: A NumPy array representing the mixed audio
- **Example**:
  - If `audio1` and `audio2` are 2 seconds long, and `amplitudes = [0.2, 0.8]`, the output is `0.2 * audio1 + 0.8 * audio2`

In [13]:
def concatenate_audio(list_of_your_audio):
    #Q4: Your code goes here
    return np.concatenate(list_of_your_audio)

In [14]:
def mix_audio(list_of_your_audio, amplitudes):
    #Q4: Your code goes here
    mixed_audio = np.zeros(len(list_of_your_audio[0]))
    for i in range(len(list_of_your_audio)):
        mixed_audio += list_of_your_audio[i] * amplitudes[i]
    return mixed_audio

5. Modify your solution to Q2 so that your pipeline can generate sawtooth waves by adding harmonics based on the following equation:

$$
    \text{sawtooth}(f, t) = \frac{2}{\pi} \sum_{k=1}^{19} \frac{(-1)^{k+1}}{k} \sin(2\pi k f t)
$$

- **Inputs**:
  - `frequency`: Fundamental frequency of sawtooth wave
  - `duration`: A float representing the duration in seconds (e.g., 2.0)
- **Output**: A NumPy array representing the audio waveform at a sample rate of 44100 Hz

In [20]:
def create_sawtooth_wave(frequency, duration, sample_rate=44100):
    #Q5: Your code goes here
    t = np.linspace(0, duration, int(duration * sample_rate), endpoint=False)
    wave = np.zeros(len(t))
    for k in range(1, 20):
        wave += (-1)**(k+1) / k * np.sin(2 * np.pi * k * frequency * t)
    wave = 2 / np.pi * wave

    return wave

## Part B - Binary Classification
Train a binary classification model using `scikit-learn` to distinguish between piano and drum MIDI files.

#### Unzip MIDI Files
Extract the provided MIDI datasets:

```bash
unzip piano.zip
unzip drums.zip
```

- `./piano`: Contains piano MIDI files (e.g., `0000.mid` to `2154.mid`)
- `./drums`: Contains drum MIDI files (e.g., `0000.mid` to `2154.mid`)
- Source: [Tegridy MIDI Dataset] (https://github.com/asigalov61/Tegridy-MIDI-Dataset)

These folders should be extracted into the same directory as your solution file

In [22]:
# NVM DON'T NEED THIS CELL

# # comment this out when submitting to gradescope
# %%capture
# # ^ otherwise this prints out a long output lol
# !unzip piano.zip
# !unzip drums.zip

6. Write functions to compute simple statistics about the files

####  `get_stats()`

- **Inputs**:
  - `piano_file_paths`: List of piano MIDI file paths`
  - `drum_file_paths`: List of drum MIDI file paths`
- **Output**: A dictionary:
  - `"piano_midi_num"`: Integer, number of piano files
  - `"drum_midi_num"`: Integer, number of drum files
  - `"average_piano_beat_num"`: Float, average number of beats in piano files
  - `"average_drum_beat_num"`: Float, average number of beats in drum files
- **Details**:
  - For each file:
    - Load with `MidiFile(file_path)`
    - Get `ticks_per_beat` from `mid.ticks_per_beat`
    - Compute total ticks as the maximum cumulative `msg.time` (delta time) across tracks
    - Number of beats = (total ticks / ticks_per_beat)
  - Compute averages, handling empty lists (return 0 if no files)

In [23]:
def get_file_lists():
    piano_files = sorted(glob.glob("./piano/*.mid"))
    drum_files = sorted(glob.glob("./drums/*.mid"))
    return piano_files, drum_files

def get_num_beats(file_path):
    # Q6: Your code goes here
    mid = MidiFile(file_path)
    # Might need: mid.tracks, msg.time, mid.ticks_per_beat

    # midi files: https://mido.readthedocs.io/en/stable/files/midi.html
    total_ticks = 0
    for track in mid.tracks:
        cumulative_time = 0
        for msg in track:
            cumulative_time += msg.time
        total_ticks = max(total_ticks, cumulative_time)
    nBeats = total_ticks / mid.ticks_per_beat

    return nBeats

def get_stats(piano_path_list, drum_path_list):
    piano_beat_nums = []
    drum_beat_nums = []
    for file_path in piano_path_list:
        piano_beat_nums.append(get_num_beats(file_path))

    for file_path in drum_path_list:
        drum_beat_nums.append(get_num_beats(file_path))

    return {"piano_midi_num":len(piano_path_list),
            "drum_midi_num":len(drum_path_list),
            "average_piano_beat_num":np.average(piano_beat_nums),
            "average_drum_beat_num":np.average(drum_beat_nums)}

7. Implement a few simple feature functions, to compute the lowest and highest MIDI note numbers in a file, and the set of unique notes in a file

`get_lowest_pitch()` and `get_highest_pitch()`
functions to find the lowest and highest MIDI note numbers in a file

- **Input**: `file_path`, a string (e.g., `"./piano/0000.mid"`)
- **Output**: An integer (0–127) or `None` if no notes exist
- **Details**:
  - Use `MidiFile(file_path)` and scan all tracks
  - Check `msg.type == 'note_on'` and `msg.velocity > 0` for active notes
  - Return the minimum (`get_lowest_pitch`) or maximum (`get_highest_pitch`) `msg.note`

`get_unique_pitch_num()`
a function to count unique MIDI note numbers in a file

- **Input**: `file_path`, a string
- **Output**: An integer, the number of unique pitches
- **Details**:
  - Collect `msg.note` from all `'note_on'` events with `msg.velocity > 0` into a set
  - Return the set’s length
- **Example**: For notes `["C4", "C4", "G4", "G4", "A4", "A4", "G4"]`, output is 3 (unique: C4, G4, A4)

In [ ]:
def get_lowest_pitch(file_path):
    #Q7-1: Your code goes here
    mid = MidiFile(file_path)
    lowest_pitch = 127 # 128-note MIDI standard range is 0-127
    for track in mid.tracks:
        for msg in track:
            if msg.type == 'note_on' and msg.velocity > 0 and msg.note < lowest_pitch:
                lowest_pitch = msg.note
    return lowest_pitch

def get_highest_pitch(file_path):
    #Q7-2: Your code goes here
    mid = MidiFile(file_path)
    highest_pitch = 0
    for track in mid.tracks:
        for msg in track:
            if msg.type == 'note_on' and msg.velocity > 0 and msg.note > highest_pitch:
                highest_pitch = msg.note
    return highest_pitch

def get_unique_pitch_num(file_path):
    #Q7-3: Your code goes here
    mid = MidiFile(file_path)
    pitches = set()
    for track in mid.tracks:
        for msg in track:
            if msg.type == 'note_on' and msg.velocity > 0:
                pitches.add(msg.note)
    return len(pitches)

8. Implement an additional feature extraction function to compute the average MIDI note number in a file

`get_average_pitch_value()`
a function to return the average MIDI note number from a file

- **Input**: `file_path`, a string
- **Output**: A float, the average value of MIDI notes in the file
- **Details**:
  - Collect `msg.note` from all `'note_on'` events with `msg.velocity > 0` into a set
- **Example**: For notes `[51, 52, 53]`, output is `52`

In [24]:
def get_average_pitch_value(file_path):
    #Q8: Your code goes here
    mid = MidiFile(file_path)
    notes = []
    for track in mid.tracks:
        for msg in track:
            if msg.type == 'note_on' and msg.velocity > 0:
                notes.append(msg.note)
    return np.average(notes)

9. Construct your dataset and split it into train and test sets using `scikit-learn` (most of this code is provided). Train your model to classify whether a given file is intended for piano or drums.

`featureQ9()`

Returns a feature vector concatenating the four features described above

- **Input**: `file_path`, a string.
- **Output**: A vector of four features

In [25]:
def featureQ9(file_path):
    # Already implemented: this one is a freebie if you got everything above correct!
    return [get_lowest_pitch(file_path),
            get_highest_pitch(file_path),
            get_unique_pitch_num(file_path),
            get_average_pitch_value(file_path)]

10. Creatively incorporate additional features into your classifier to make your classification more accurate.  Include comments describing your solution.

In [26]:
# I get the impression that drum pieces are a lot faster than
# piano pieces so they might have a higher average notes/sec
def get_average_notes_per_sec(file_path):
  mid = MidiFile(file_path)
  num_notes = 0
  for track in mid.tracks:
    for msg in track:
      if msg.type == 'note_on' and msg.velocity > 0:
        num_notes += 1
  return num_notes / mid.length

# Drum pieces feel a lot more intense and loud than piano pieces,
# so this could be helpful information since higher average
# note velocity could indicate drums
def get_average_note_velocity(file_path):
  mid = MidiFile(file_path)
  num_notes = 0
  total_velocity = 0
  for track in mid.tracks:
    for msg in track:
      if msg.type == 'note_on' and msg.velocity > 0:
        num_notes += 1
        total_velocity += msg.velocity
  return total_velocity / num_notes

def featureQ10(file_path):
    #Q10: Your code goes here
    return [get_lowest_pitch(file_path),
            get_highest_pitch(file_path),
            get_unique_pitch_num(file_path),
            get_average_pitch_value(file_path),
            get_average_notes_per_sec(file_path),
            get_average_note_velocity(file_path)]